<a href="https://colab.research.google.com/github/HarveCriqui/Colab/blob/main/BookTranslator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import requests
!pip install ebooklib #transformers sentencepiece sacremoses
!pip install -q openai
import ebooklib
import requests
import re
from ebooklib import epub
from bs4 import BeautifulSoup
import openai

openai.api_key = ''

# For adding to current book
book = epub.read_epub('/content/drive/MyDrive/LightNovels/Books/EvolutionaryNecromancer1.epub')

# For new book
#book = epub.EpubBook()

cover_image_path = '/content/drive/MyDrive/LightNovels/pngs/Necromancers_Evolution.png'

cover_item = epub.EpubItem(uid="cover-image", file_name="cover.jpg", content=open(cover_image_path, "rb").read())

# Add the cover image to the book's manifest
book.add_item(cover_item)

# Add initial spine and toc

#book.toc = []
#book.spine = []

# Set Meta Data
book.set_identifier("strongestNecromancer")
book.set_title("The Strongest Necromancer With Evolution Traits")
book.set_language("en")
book.add_author("Chabindong Ghost")


def split_string_three(text):
    left_index = len(text) // 3
    middle_index = len(text) * 2 // 3
    left_third = text[:left_index]
    middle_third = text[left_index:middle_index]

    # Find the nearest space to the left of the middle index
    last_space_index = left_third.rfind('\n')
    last_space_index2 = middle_third.rfind('\n')

    if last_space_index != -1 and last_space_index2 != -1:
        # Split the string at the found space
        left_third = text[:last_space_index]
        middle_third = text[last_space_index + 1:middle_index]
        right_third = text[middle_index + 1:]
    else:
        # If no space found in either third, just split the string in half
        left_third = text[:left_index]
        middle_third = text[left_index:middle_index]
        right_third = text[middle_index:]

    return left_third, middle_third, right_third

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=e02a7250effc3db9baefa14c38b4b2037c12b9f8dbfed21dff6428567a202485
  Stored in directory: /root/.cache/pip/wheels/0f/38/cc/a3728bb72a315d9d8766fb71d362136372066fc25ad838f8fa
Successfully built ebooklib
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


/usr/local/lib/python3.10/dist-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [3]:
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        print('==================================')
        print('NAME : ', item.get_name())
        print('----------------------------------')
        print(item.get_content())
        print('==================================')

NAME :  chap_1.xhtml
----------------------------------
b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">\n  <head/>\n  <body><h1>Chapter 1.</h1>\n    <br/>\n    <p>Adrias Cromwell (1)</p>\n    <br/>\n    <p>\n[Quest: Clear the End of Baal!]</p>\n    <br/>\n    <p>[Free play is now available.]</p>\n    <br/>\n    <p>[A new character has been unlocked!]</p>\n    <br/>\n    <p>\nI let go of the mouse, ignoring the message that I\'ve seen over 10 times already.</p>\n    <br/>\n    <p>\nThud.</p>\n    <br/>\n    <p>I stretched my body as hard as I could, and my whole body screamed in relief.</p>\n    <br/>\n    <p>\nWhen I checked the time, it was 11 AM.</p>\n    <br/>\n    <p>\nEven though I stayed up all night, I felt fine.</p>\n    <br/>\n    <p>\nThump.</p>\n    <br/>\n    <p>Thump.</p>\n    <br

In [ ]:
'''book = epub.read_epub('/content/drive/MyDrive/LightNovels/Books/EvolutionaryNecromancer.epub')
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        print('==================================')
        print('NAME : ', item.get_name())
        print('----------------------------------')
        print(item.get_content())
        print('==================================')'''
book.toc

In [ ]:
#book = epub.read_epub('/content/drive/MyDrive/LightNovels/Books/EvolutionaryNecromancer2.epub')

In [8]:
# Initial URL
URL = f"https://agit573.xyz/novel/view/991267/2"
curr_chap = 1
# Create a loop || range(1,100) is chapters 1 to 99
for i in range(15,20):

# Set URL to first chapter in range
  while curr_chap < i:
    page = requests.get(URL, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'})
    soup = BeautifulSoup(page.content, "html.parser")
    link = soup.find('a', string='다음화')
    number_match = re.search(r"fn_goto_page\('next', (\d+)\);", link.get('href'))
    next_url = f"https://agit573.xyz/novel/view/{number_match.group(1)}/2"
    URL = next_url
    curr_chap += 1
    print(URL)
  page = requests.get(URL, headers={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'})
  soup = BeautifulSoup(page.content, "html.parser")
  results = soup.find(id="id_wr_content")
  curr_chap += 1
# Split chapter into 3rd to make sure that it doesn't overload on tokens

  left, middle, right = split_string_three(results.getText(separator = '\n', strip = True))

# Change URL to next chapter

  link = soup.find('a', string='다음화')
  number_match = re.search(r"fn_goto_page\('next', (\d+)\);", link.get('href'))
  next_url = f"https://agit573.xyz/novel/view/{number_match.group(1)}/2"
  URL = next_url

# Send 3 requests to Chat GPT

  saved = []
  for text in [left, middle, right]:
    messages = [ {"role": "system", "content":
                  "You are translating a korean web novel to english"} ]
    messages.append(
        {"role": "user", "content": text},
    )
    chat = openai.ChatCompletion.create(
        model="gpt-3.5-turbo", messages=messages, temperature=0.5
    )
    reply = chat.choices[0].message.content
    saved.append(reply)

  # Stitch chapter back together

  chapter_text = f"{saved[0]}\n{saved[1]}\n{saved[2]}"

  # Remove all newlines

  chapter_text.replace('/n', ' ')
  sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!|\]|"|\)|\*)\s', chapter_text)

  titled = sentences[0]
  sentences.pop(0)

  for sen in sentences:
    sen = sen.strip(' ')
  # Join the sentences back with newline characters
  formatted_text = '</p><br/><p>'.join(sentences)

  # Create chapter object and add it to epub file

  c1 = epub.EpubHtml(title=titled, file_name=f"chap_{i}.xhtml", uid=f'chap_{i}', lang="en")
  c1.content = (f"<h1>{titled}</h1><br/><p>{formatted_text}</p>")
  c2 = epub.Link(f'chap_{i}.xhtml', f'Chapter {i}', f'chap_{i}')
  book.add_item(c1)
  book.toc.append(c2)
  book.spine.append(f'chap_{i}')

https://agit573.xyz/novel/view/991272/2
https://agit573.xyz/novel/view/991277/2
https://agit573.xyz/novel/view/991282/2
https://agit573.xyz/novel/view/991287/2
https://agit573.xyz/novel/view/991292/2
https://agit573.xyz/novel/view/991297/2
https://agit573.xyz/novel/view/991302/2
https://agit573.xyz/novel/view/991307/2
https://agit573.xyz/novel/view/991312/2


In [9]:
#book2 = epub.read_epub('/content/drive/MyDrive/LightNovels/Books/EvolutionaryNecromancer2.epub')
book.toc

In [10]:
for item in book.items:
    if item.id == 'nav':
        book.items.remove(item)
        break

book.add_item(epub.EpubNcx())
book.add_item(epub.EpubNav())
for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
        print('==================================')
        print('NAME : ', item.get_name())
        print('----------------------------------')
        print(item.get_content())
        print('==================================')

NAME :  chap_1.xhtml
----------------------------------
b'<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:epub="http://www.idpf.org/2007/ops" epub:prefix="z3998: http://www.daisy.org/z3998/2012/vocab/structure/#" lang="en" xml:lang="en">\n  <head/>\n  <body><h1>Chapter 1.</h1>\n    <br/>\n    <p>Adrias Cromwell (1)</p>\n    <br/>\n    <p>\n[Quest: Clear the End of Baal!]</p>\n    <br/>\n    <p>[Free play is now available.]</p>\n    <br/>\n    <p>[A new character has been unlocked!]</p>\n    <br/>\n    <p>\nI let go of the mouse, ignoring the message that I\'ve seen over 10 times already.</p>\n    <br/>\n    <p>\nThud.</p>\n    <br/>\n    <p>I stretched my body as hard as I could, and my whole body screamed in relief.</p>\n    <br/>\n    <p>\nWhen I checked the time, it was 11 AM.</p>\n    <br/>\n    <p>\nEven though I stayed up all night, I felt fine.</p>\n    <br/>\n    <p>\nThump.</p>\n    <br/>\n    <p>Thump.</p>\n    <br

In [11]:
# create book
epub.write_epub('/content/drive/MyDrive/LightNovels/Books/EvolutionaryNecromancer1.epub', book)